# **Animated Choropleth**

This notebook demonstrates an animated choropleth where each frame represents a year in a time series. The notebook loads a polygon GeoJSON dataset and produces a choropleth visualisation.

The key parameters need to be edited in the Parameters section below, in particular for selecting the data of interest. 

In [90]:
#  Load the required libraries

library(sf)
library(ggplot2)
library(classInt)
library(plotly)

#library(gganimate)
#library(transformr)

##  Parameters

Edit the key parameters here; the dataset, the varaible of interest and the number of categories for the variable to be classified into. 

In [91]:
#  Insert the dataset name and variable of interest

## Dataset
dataset1.param <- "ABS_Data-by-Region_Education-Employment_SA2_2014.geojson"
dataset2.param <- "ABS_Data-by-Region_Education-Employment_SA2_2015.geojson"
dataset3.param <- "ABS_Data-by-Region_Education-Employment_SA2_2016.geojson"
dataset4.param <- "ABS_Data-by-Region_Education-Employment_SA2_2017.geojson"


## Variable of Interest
variable.param <- "jobs_australia_year_ended_30_june_number_employee_jobs_total"

## Number of Categories
intervals.param <- 6

## Load Dataset

In [92]:
#  The data are loaded here and converted to an sf dataframe.

getwd()
setwd("/Users/greenwood/Documents/workspace/juno-tools")                                                    # Set the working directory
getwd()

dataset1.sf <-  st_read(paste("data/animated-choropleth/", dataset1.param, sep = ""))                # Load the GeoJSON dataset
dataset2.sf <-  st_read(paste("data/animated-choropleth/", dataset2.param, sep = ""))                # Load the GeoJSON dataset
dataset3.sf <-  st_read(paste("data/animated-choropleth/", dataset3.param, sep = ""))                # Load the GeoJSON dataset
dataset4.sf <-  st_read(paste("data/animated-choropleth/", dataset4.param, sep = ""))                # Load the GeoJSON dataset

dataset.sf <- rbind(dataset1.sf, dataset2.sf, dataset3.sf, dataset4.sf)

[1] "/Users/greenwood/Documents/workspace/juno-tools"

[1] "/Users/greenwood/Documents/workspace/juno-tools"

Reading layer `ABS_Data-by-Region_Education-Employment_SA2_2014' from data source `/Users/greenwood/Documents/workspace/juno-tools/data/animated-choropleth/ABS_Data-by-Region_Education-Employment_SA2_2014.geojson' using driver `GeoJSON'
Simple feature collection with 306 features and 7 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 144.3336 ymin: -38.50299 xmax: 145.8784 ymax: -37.1751
CRS:            4283
Reading layer `ABS_Data-by-Region_Education-Employment_SA2_2015' from data source `/Users/greenwood/Documents/workspace/juno-tools/data/animated-choropleth/ABS_Data-by-Region_Education-Employment_SA2_2015.geojson' using driver `GeoJSON'
Simple feature collection with 307 features and 7 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 144.3336 ymin: -38.50299 xmax: 145.8784 ymax: -37.1751
CRS:            4283
Reading layer `ABS_Data-by-Region_Education-Employment_SA2_2016' from data source `/Users/greenwood/Documents/workspace/

## Prepare Data

This cell creates a new variable in the dataset that is a categorisation of the variable of interest into a series of intervals.

In [93]:
# Generate a list of intervals
cint <- classIntervals(dataset.sf[[variable.param]], intervals.param,
                       style = "quantile")                                     # Set the type of classifcation, e.g. "equal" for equal intervals

print(cint)
                                                                        
# Create the categorised variable according to the intervals generated above 
dataset.sf$choropleth_cat <- cut(dataset.sf[[variable.param]], 
                                 breaks = data.frame(cint[2])[,1],
                                 include.lowest = TRUE,
                                 dig.lab = 7)                                  # dig.lab sets the number of digits in the label before scientific notation is used

Warning message in classIntervals(dataset.sf[[variable.param]], intervals.param, :
“var has missing values, omitted in finding classes”


style: quantile
       [5,6005.667) [6005.667,7898.667)     [7898.667,9956)        [9956,12131) 
                205                 204                 204                 205 
      [12131,14697)       [14697,41309] 
                204                 205 


## Plot Visualisation

This cell contains the code for plotting the choropleth using `ggplot()`. See https://ggplot2.tidyverse.org/ for further information.

In [94]:
p <- ggplot(data = dataset.sf, aes(frame = yr))+
  geom_sf(aes(fill=dataset.sf$choropleth_cat),colour='grey', size = 0.2, data=dataset.sf)+
  scale_fill_viridis_d(                                                        # Set the colour scheme here. See https://ggplot2.tidyverse.org/reference/scale_viridis.html
    name='Legend',                                        # Set the title of the legend
    guide=guide_legend(
      direction='vertical',                                 # Set the orientation of the legend
      title.position='top',                                 # Set the postion of the legend title
      title.hjust = .5,                                     # Adjust horizontal postion of the legend title
      label.hjust = .5,                                     # Adjust horizontal postion of the legend labels
      label.position = 'right',                             # Position of labels Change to e.g. 'bottom'
      keywidth = 1,                                         # Legend width
      keyheight = 1                                         # Legend height
    ))+
  labs(title="Choropleth Map",                                                 # Map title
       subtitle= paste('Map of Variable', variable.param),                     # Map subtitle
       caption=c('Source: AURIN aurin.org.au'))+                               # Caption
  theme_void()+
  theme(title=element_text(face='bold'),
        legend.position = 'right')                                             # Position of legend. Change to e.g. 'bottom'


#m <- list(
#  l = 50,
#  r = 50,
#  b = 100,
#  t = 1000,
#  pad = 4
#)
#fig <- fig %>% layout(autosize = F, width = 500, height = 500, margin = m)

In [ ]:
fig <- ggplotly(p)
fig

In [96]:
#pa <- p + transition_time(yr) +
#  labs(title = "Year: {frame_time}")

In [97]:
#library(gifski)
#ani <- pa + exit_shrink()
#animate(ani, duration = 5, fps = 20, width = 200, height = 200, renderer = gifski_renderer())
#anim_save("output.gif",animation = ani)

## Save Result

In [98]:
htmlwidgets::saveWidget(fig, "animated-choropleth.html")

Warning message in p$x$data[firstFrame] <- p$x$frames[[1]]$data:
“number of items to replace is not a multiple of replacement length”
